In [20]:
import os
%pwd

'c:\\Users\\abdus samad\\Desktop\\Churn_ML\\Churn_ML_Project'

In [21]:
os.chdir("c:\\Users\\abdus samad\\Desktop\\Churn_ML\\Churn_ML_Project")
%pwd

'c:\\Users\\abdus samad\\Desktop\\Churn_ML\\Churn_ML_Project'

In [22]:
from dataclasses import dataclass
from pathlib import Path

In [ ]:
@dataclass(frozen=True)
class ModelTunerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    target_column: str
    n_trails: int
    study_name: str
    best_params_path: Path
    mlflow_uri: str

In [50]:
from src.Churn_Predictor.constants import *
from src.Churn_Predictor.utils.common import create_directories, read_yaml

In [65]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH
                 ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_model_tuner_config(self) -> ModelTunerConfig:
        config = self.config.model_tuner
        target_column = list(self.schema.TARGET_COLUMN.keys())[0] 
        create_directories([config.root_dir])

        model_tuner_config = ModelTunerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            target_column=target_column,
            n_trails=config.n_trails,
            study_name=config.study_name,
            best_params_path=config.best_params_path,
            mlflow_uri=os.getenv('MLFLOW_TRACKING_URI', 'file:./mlruns')  
        )
        print(model_tuner_config)
        return model_tuner_config

In [66]:
import os
import pandas as pd
import yaml
import mlflow
import optuna
from pathlib import Path
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from dotenv import load_dotenv
from src.Churn_Predictor import logger

load_dotenv()


class ModelTuner:
    def __init__(self, config: ModelTunerConfig):
        self.config = config
        
    def objective(self, trial, X_train, y_train, X_val, y_val):
        """
        Objective function for Optuna to optimize
        
        trial: Optuna trial object
        Returns: F1 score (to be maximized)
        """
        # Define parameter search space
        params = {
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'n_estimators': trial.suggest_int('n_estimators', 50, 300),
            'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1, 5),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 7),
            'gamma': trial.suggest_float('gamma', 0, 0.5),
            'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
            'reg_lambda': trial.suggest_float('reg_lambda', 0, 1)
        }
        
        # Train model with suggested parameters
        model = XGBClassifier(
            **params,
            random_state=42,
            use_label_encoder=False,
            eval_metric='logloss',
            early_stopping_rounds=10,
            verbosity=0  # Suppress XGBoost output
        )
        
        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            verbose=False
        )
        
        # Predict and calculate F1 score
        y_pred = model.predict(X_val)
        f1 = f1_score(y_val, y_pred)
        
        return f1
    
    def tune_hyperparameters(self):
        """
        Main tuning method using Optuna
        """
        logger.info("Starting hyperparameter tuning with Optuna")
        
        # Load data
        logger.info(f"Loading training data from: {self.config.train_data_path}")
        train_data = pd.read_csv(self.config.train_data_path)
        
        # Split features and target
        X = train_data.drop(columns=[self.config.target_column])
        y = train_data[self.config.target_column]
        
        logger.info(f"Dataset shape: {X.shape}")
        logger.info(f"Target distribution:\n{y.value_counts()}")
        
        # Split into train and validation sets
        X_train, X_val, y_train, y_val = train_test_split(
            X, y, 
            test_size=0.2, 
            random_state=42, 
            stratify=y
        )
        
        logger.info(f"Train set: {X_train.shape}, Validation set: {X_val.shape}")
        
        # Setup MLflow
        mlflow.set_tracking_uri(self.config.mlflow_uri)
        mlflow.set_experiment("Churn_Prediction_Optuna_Tuning")
        
        # Setup MLflow callback for Optuna
        from optuna.integration.mlflow import MLflowCallback
        
        mlflc = MLflowCallback(
            tracking_uri=self.config.mlflow_uri,
            metric_name="f1_score"
        )
        
        # Create Optuna study
        logger.info(f"Creating Optuna study: {self.config.study_name}")
        study = optuna.create_study(
            direction='maximize',  # Maximize F1 score
            study_name=self.config.study_name,
            sampler=optuna.samplers.TPESampler(seed=42)  # Tree-structured Parzen Estimator
        )
        
        # Run optimization
        logger.info(f"Starting optimization with {self.config.n_trails} trials...")
        logger.info("This may take a while...")
        
        study.optimize(
            lambda trial: self.objective(trial, X_train, y_train, X_val, y_val),
            n_trials=self.config.n_trails,
            callbacks=[mlflc],
            show_progress_bar=True
        )
        
        # Get best parameters
        best_params = study.best_params
        best_score = study.best_value
        
        logger.info("=" * 70)
        logger.info("OPTIMIZATION COMPLETED")
        logger.info("=" * 70)
        logger.info(f"Best trial number: {study.best_trial.number}")
        logger.info(f"Best F1 score: {best_score:.4f}")
        logger.info(f"Best parameters:")
        for param, value in best_params.items():
            logger.info(f"  {param}: {value}")
        logger.info("=" * 70)
        
        # Save best parameters to YAML
        self._save_best_params(best_params)
        
        # Generate optimization visualizations
        self._generate_visualizations(study)
        
        return best_params, best_score
    
    def _save_best_params(self, best_params):
        """Save best parameters to YAML file"""
        # Format parameters for YAML (matching params.yaml structure)
        yaml_params = {
            'XGBBoost': {
                'n_estimators': int(best_params['n_estimators']),
                'max_depth': int(best_params['max_depth']),
                'learning_rate': float(best_params['learning_rate']),
                'scale_pos_weight': float(best_params['scale_pos_weight']),
                'subsample': float(best_params['subsample']),
                'colsample_bytree': float(best_params['colsample_bytree']),
                'min_child_weight': int(best_params['min_child_weight']),
                'gamma': float(best_params['gamma']),
                'reg_alpha': float(best_params['reg_alpha']),
                'reg_lambda': float(best_params['reg_lambda']),
                'random_state': 42
            }
        }
        
        # Save to file
        with open(self.config.best_params_path, 'w') as f:
            yaml.dump(yaml_params, f, default_flow_style=False)
        
        logger.info(f"Best parameters saved to: {self.config.best_params_path}")
        logger.info("You can now copy these to params.yaml to use them in training!")
    
    def _generate_visualizations(self, study):
        """Generate Optuna visualization plots"""
        try:
            from optuna.visualization import (
                plot_optimization_history,
                plot_param_importances,
                plot_slice
            )
                        
            # Create visualizations directory
            viz_dir = os.path.join(self.config.root_dir, 'visualizations')
            os.makedirs(viz_dir, exist_ok=True)
            
            # 1. Optimization history
            fig1 = plot_optimization_history(study)
            fig1.write_html(os.path.join(viz_dir, 'optimization_history.html'))
            
            # 2. Parameter importances
            fig2 = plot_param_importances(study)
            fig2.write_html(os.path.join(viz_dir, 'param_importances.html'))
            
            # 3. Slice plot
            fig3 = plot_slice(study)
            fig3.write_html(os.path.join(viz_dir, 'param_slice.html'))
            
            logger.info(f"Visualizations saved to: {viz_dir}")
            
        except Exception as e:
            logger.warning(f"Could not generate visualizations: {e}")

In [67]:
try:
    config = ConfigurationManager()
    model_tuner_config = config.get_model_tuner_config()
    model_tuner = ModelTuner(config=model_tuner_config)
    best_params, best_score = model_tuner.tune_hyperparameters()

    logger.info(f"Tuning Completed! Best F1 score: {best_score:.4f}")
    print(f"The Best Params: {best_params}: {best_score}")
except Exception as e:
    logger.error("Error")
    raise e

[2026-02-14 23:32:26,150: INFO: common: yaml file: config\config.yaml loaded successfully]
[2026-02-14 23:32:26,152: INFO: common: yaml file: params.yaml loaded successfully]
[2026-02-14 23:32:26,156: INFO: common: yaml file: schema.yaml loaded successfully]
[2026-02-14 23:32:26,158: INFO: common: created directory at: artifacts]
[2026-02-14 23:32:26,159: INFO: common: created directory at: artifacts/model_tuner]
ModelTunerConfig(root_dir='artifacts/model_tuner', train_data_path='artifacts/data_transformation/train.csv', test_data_path='artifacts/data_transformation/test.csv', target_column='Churn', n_trails=100, study_name='churn_prediction_optuna', best_params_path='artifacts/model_tuner/best_params.yaml', mlflow_uri='https://dagshub.com/Maazthepal/Churn_ML_Project.mlflow')
[2026-02-14 23:32:26,161: INFO: 4061246130: Starting hyperparameter tuning with Optuna]
[2026-02-14 23:32:26,162: INFO: 4061246130: Loading training data from: artifacts/data_transformation/train.csv]
[2026-02-14 

C:\Users\abdus samad\AppData\Local\Temp\ipykernel_16904\4061246130.py:97: ExperimentalWarning: MLflowCallback is experimental (supported from v1.4.0). The interface can change in the future.
  mlflc = MLflowCallback(
[I 2026-02-14 23:32:28,719] A new study created in memory with name: churn_prediction_optuna


[2026-02-14 23:32:28,720: INFO: 4061246130: Starting optimization with 100 trials...]
[2026-02-14 23:32:28,721: INFO: 4061246130: This may take a while...]


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2026-02-14 23:32:29,379] Trial 0 finished with value: 0.6506746626686657 and parameters: {'learning_rate': 0.03574712922600244, 'max_depth': 10, 'n_estimators': 233, 'scale_pos_weight': 3.3946339367881464, 'subsample': 0.6624074561769746, 'colsample_bytree': 0.662397808134481, 'min_child_weight': 1, 'gamma': 0.4330880728874676, 'reg_alpha': 0.6011150117432088, 'reg_lambda': 0.7080725777960455}. Best is trial 0 with value: 0.6506746626686657.
🏃 View run 0 at: https://dagshub.com/Maazthepal/Churn_ML_Project.mlflow/#/experiments/2/runs/2470b45f1d7e47dab57ec6820a908182
🧪 View experiment at: https://dagshub.com/Maazthepal/Churn_ML_Project.mlflow/#/experiments/2


...

Best trial: 31. Best value: 0.664625:  99%|█████████▉| 99/100 [11:58<00:07,  7.71s/it]

[I 2026-02-14 23:44:27,671] Trial 99 finished with value: 0.6335078534031413 and parameters: {'learning_rate': 0.0619064160795348, 'max_depth': 6, 'n_estimators': 129, 'scale_pos_weight': 4.533477308902299, 'subsample': 0.64791974872656, 'colsample_bytree': 0.60122315623032, 'min_child_weight': 3, 'gamma': 0.227000776013018, 'reg_alpha': 0.308476413156403, 'reg_lambda': 0.2542707871251489}. Best is trial 31 with value: 0.664624808575804.
🏃 View run 99 at: https://dagshub.com/Maazthepal/Churn_ML_Project.mlflow/#/experiments/2/runs/f026a2636d9a4202a30f2e85820ad649
🧪 View experiment at: https://dagshub.com/Maazthepal/Churn_ML_Project.mlflow/#/experiments/2


Best trial: 31. Best value: 0.664625: 100%|██████████| 100/100 [12:02<00:00,  7.22s/it]

[2026-02-14 23:44:30,882: INFO: 4061246130: ======================================================================]
[2026-02-14 23:44:30,884: INFO: 4061246130: OPTIMIZATION COMPLETED]
[2026-02-14 23:44:30,885: INFO: 4061246130: ======================================================================]
[2026-02-14 23:44:30,887: INFO: 4061246130: Best trial number: 31]
[2026-02-14 23:44:30,888: INFO: 4061246130: Best F1 score: 0.6646]
[2026-02-14 23:44:30,889: INFO: 4061246130: Best parameters:]
[2026-02-14 23:44:30,890: INFO: 4061246130:   learning_rate: 0.1055633400893983]
[2026-02-14 23:44:30,892: INFO: 4061246130:   max_depth: 7]
[2026-02-14 23:44:30,893: INFO: 4061246130:   n_estimators: 190]
[2026-02-14 23:44:30,895: INFO: 4061246130:   scale_pos_weight: 2.136068784342291]
[2026-02-14 23:44:30,896: INFO: 4061246130:   subsample: 0.679490888668883]
[2026-02-14 23:44:30,898: INFO: 4061246130:   colsample_bytree: 0.6440613131985963]
[2026-02-14 23:44:30,899: INFO: 4061246130:   min_child

[2026-02-14 23:44:33,944: WARNING: 4061246130: Could not generate visualizations: Tried to import 'plotly' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named 'plotly'.]
[2026-02-14 23:44:33,945: INFO: 2842814328: Tuning Completed! Best F1 score: 0.6646]
The Best Params: {'learning_rate': 0.1055633400893983, 'max_depth': 7, 'n_estimators': 190, 'scale_pos_weight': 2.136068784342291, 'subsample': 0.679490888668883, 'colsample_bytree': 0.6440613131985963, 'min_child_weight': 6, 'gamma': 0.47448062682421965, 'reg_alpha': 0.5538326471886972, 'reg_lambda': 0.2239251831306163}: 0.664624808575804
